In [ ]:
# bibliotecas

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import KFold
import joblib

In [ ]:
df = pd.read_csv('/content/State_of_data_2022.csv')
df = df.drop_duplicates()

In [ ]:

df.rename(columns={ # colunas sobre a identidade dos colaboradores de 2022
    "('P0', 'id')" : "id",
    "('P1_a_1 ', 'Faixa idade')" : "idade",
    "('P1_b ', 'Genero')" : "genero",
    "('P1_c ', 'Cor/raca/etnia')" : "cor",
    "('P1_d ', 'PCD')" : 'pcd',
    "('P1_e ', 'experiencia_profissional_prejudicada')" : "experiencia_prejudicada",
    "('P1_i_2 ', 'Regiao onde mora')" : 'regiao',
    "('P1_l ', 'Nivel de Ensino')" : "ensino",
    "('P1_m ', 'Área de Formação')" : "area_de_formacao",
    "('P2_a ', 'Qual sua situação atual de trabalho?')" : "situacao_atual",
    "('P2_b ', 'Setor')" : "setor",
    "('P2_c ', 'Numero de Funcionarios')" : 'numero_de_funcionarios',
    "('P2_d ', 'Gestor?')" : "gestor",
    "('P2_f ', 'Cargo Atual')": "cargo_atual",
    "('P2_g ', 'Nivel')" : "nivel",
    "('P2_h ', 'Faixa salarial')": "faixa_salario",
    "('P2_i ', 'Quanto tempo de experiência na área de dados você tem?')": "experiencia_com_dados",
    "('P2_j ', 'Quanto tempo de experiência na área de TI/Engenharia de Software você teve antes de começar a trabalhar na área de dados?')" : "experiencia_com_TI_engenharia",
    "('P2_k ', 'Você está satisfeito na sua empresa atual?')" : "satisfeito",

    # Colunas sobre modelo de trabalho remoto de colaboradores de 2022
    "('P2_p ', 'Atualmente qual a sua forma de trabalho?')" : 'modelo_de_trabalho',
    "('P2_q ', 'Qual a forma de trabalho ideal para você?')" : "modelo_de_trabalho_ideal",
    "('P2_r ', 'Caso sua empresa decida pelo modelo 100% presencial qual será sua atitude?')" : "volta_ao_presencial",


    # Coluna sobre querer mudar de emprego
    "('P2_n ', 'Você pretende mudar de emprego nos próximos 6 meses?')" : "mudar_de_emprego",

    # Colunas sobre os motivos de insatisfação de colaboradores de 2022
    "('P2_l_1 ', 'Falta de oportunidade de crescimento no emprego atual')" : "oportunidade",
    "('P2_l_2 ', 'Salário atual não corresponde ao mercado')" : "salario_mercado",
    "('P2_l_3 ', 'Não tenho uma boa relação com meu líder/gestor')" : "lider",
    "('P2_l_4 ', 'Gostaria de trabalhar em em outra área de atuação')" : "area",
    "('P2_l_5 ', 'Gostaria de receber mais benefícios')" : "beneficios",
    "('P2_l_6 ', 'O clima de trabalho/ambiente não é bom')": "clima",
    "('P2_l_7 ', 'Falta de maturidade analítica na empresa')" : 'maturidade',
    "('P3_c_1 ', 'Pensar na visão de longo prazo de dados da empresa e fortalecimento da cultura analítica da companhia.')" : "visao",

    # tamnho da equipe
    "('P3_a ', 'Qual o número aproximado de pessoas que atuam com dados na sua empresa hoje?')" : "tamanho_equipe",

    # Colunas sobre desafios e funções dos gestores de 2022
    "('P3_c_2 ', 'Organização de treinamentos e iniciativas com o objetivo de aumentar a maturidade analítica das áreas de negócios.')" : "treinamento",
    "('P3_c_3 ', 'Atração, seleção e contratação de talentos para o time de dados.')" : "rh",
    "('P3_c_4 ', 'Decisão sobre contratação de ferramentas e tecnologias relacionadas a dados.')" : "ferramentas",
    "('P3_c_5 ', 'Sou gestor da equipe responsável pela engenharia de dados e por manter o Data Lake da empresa como fonte única dos dados, garantindo a qualidade e confiabilidade da informação.')" : "data_lake",
    "('P3_c_6 ', 'Sou gestor da equipe responsável pela entrega de dados, estudos, relatórios e dashboards para as áreas de negócio da empresa.')" : "gestor_equipe",
    "('P3_c_7 ', 'Sou gestor da equipe responsável por iniciativas e projetos envolvendo Inteligência Artificial e Machine Learning.')" : "gestor_ia",
    "('P3_c_8 ', 'Apesar de ser gestor ainda atuo na parte técnica, construindo soluções/análises/modelos etc.')": "parte_tecnica",
    "('P3_c_9 ', 'Gestão de projetos de dados, cuidando das etapas, equipes envolvidas, atingimento dos objetivos etc.')": "gestor_etapa",
    "('P3_c_10 ', 'Gestão de produtos de dados, cuidando da visão dos produtos, backlog, feedback de usuários etc.')": "gestor_produto",
    "('P3_c_11 ', 'Gestão de pessoas, apoio no desenvolvimento das pessoas, evolução de carreira')": "desenvolvimento",
    "('P3_d ', 'Quais são os 3 maiores desafios que você tem como gestor no atual momento?')" : "desafios_gestor"
    },inplace=True)

In [ ]:
# Separando as colunas que serão usadas
df_ml = df[["idade", "genero", "cor","ensino", 'modelo_de_trabalho','numero_de_funcionarios','tamanho_equipe', "faixa_salario",
    "experiencia_com_dados", "satisfeito" ]].dropna()

# Excluindo quem não informa
lista_apagar = list(df_ml.query("genero == 'Prefiro não informar' or cor == 'Prefiro não informar' or ensino == 'Prefiro não informar'").index)
df_ml = df_ml.drop(lista_apagar)

# Pegando os valores dummies
df_ml = pd.get_dummies(df_ml, columns=["idade", "genero", "cor","modelo_de_trabalho","ensino", 'numero_de_funcionarios', "faixa_salario",
    "experiencia_com_dados", "tamanho_equipe", "satisfeito"])

# dando uma nova ordem para o DataFrame
df_ml = df_ml[[

    #Idade

    'idade_17-21',
    'idade_22-24',
    'idade_25-29',
    'idade_30-34',
    'idade_35-39',
    'idade_40-44',
    'idade_45-49',
    'idade_50-54',
    'idade_55+',

    # Sexo

    'genero_Feminino',
    'genero_Masculino',

    # Cor / Raça / Etnia

    'cor_Amarela',
    'cor_Branca',
    'cor_Indígena',
    'cor_Parda',
    'cor_Preta',
    'cor_Outra',

    # Modelo de Trabalho

    'modelo_de_trabalho_Modelo 100% presencial',
    'modelo_de_trabalho_Modelo 100% remoto',
    'modelo_de_trabalho_Modelo híbrido com dias fixos de trabalho presencial',
    'modelo_de_trabalho_Modelo híbrido flexível (o funcionário tem liberdade para escolher quando estar no escritório presencialmente)',

    # Ensino

    'ensino_Não tenho graduação formal',
    'ensino_Estudante de Graduação',
    'ensino_Graduação/Bacharelado',
    'ensino_Pós-graduação',
    'ensino_Mestrado',
    'ensino_Doutorado ou Phd',

    # Número de Funcionários
   'numero_de_funcionarios_de 1 a 5',
   'numero_de_funcionarios_de 6 a 10',
   'numero_de_funcionarios_de 11 a 50',
   'numero_de_funcionarios_de 51 a 100',
   'numero_de_funcionarios_de 101 a 500',
   'numero_de_funcionarios_de 501 a 1.000',
   'numero_de_funcionarios_de 1.001 a 3.000',
   'numero_de_funcionarios_Acima de 3.000',

    # Equipe de Dados

    'tamanho_equipe_Ainda não temos pessoas atuando com dados na empresa',
    'tamanho_equipe_1 - 3',
    'tamanho_equipe_4 - 10',
    'tamanho_equipe_11 - 20',
    'tamanho_equipe_21 - 50',
    'tamanho_equipe_51 - 100',
    'tamanho_equipe_101 - 300',
    'tamanho_equipe_Acima de 300 pessoas',

     # Salário

    'faixa_salario_Menos de R$ 1.000/mês',
    'faixa_salario_de R$ 1.001/mês a R$ 2.000/mês',
    'faixa_salario_de R$ 2.001/mês a R$ 3.000/mês',
    'faixa_salario_de R$ 3.001/mês a R$ 4.000/mês',
    'faixa_salario_de R$ 4.001/mês a R$ 6.000/mês',
    'faixa_salario_de R$ 6.001/mês a R$ 8.000/mês',
    'faixa_salario_de R$ 8.001/mês a R$ 12.000/mês',
    'faixa_salario_de R$ 12.001/mês a R$ 16.000/mês',
    'faixa_salario_de R$ 16.001/mês a R$ 20.000/mês',
    'faixa_salario_de R$ 20.001/mês a R$ 25.000/mês',
    'faixa_salario_de R$ 25.001/mês a R$ 30.000/mês',
    'faixa_salario_de R$ 30.001/mês a R$ 40.000/mês',
    'faixa_salario_Acima de R$ 40.001/mês',

    # Experiencia com Dados

    'experiencia_com_dados_Não tenho experiência na área de dados',
    'experiencia_com_dados_Menos de 1 ano',
    'experiencia_com_dados_de 1 a 2 anos',
    'experiencia_com_dados_de 3 a 4 anos',
    'experiencia_com_dados_de 4 a 6 anos',
    'experiencia_com_dados_de 7 a 10 anos',
    'experiencia_com_dados_Mais de 10 anos',

    # Satisfeito
    'satisfeito_True'
               ]]


In [ ]:
# exportando o df
df_ml.to_csv('ml_state_of_data2022.csv', index=False)

# Separando os valores para X (variáveis independentes) e y (variável dependente)
X = df_ml.drop('satisfeito_True', axis=1).reset_index(drop=True)
y = df_ml[['satisfeito_True']].reset_index(drop=True)

# Estabelecendo modelo de ML
lr = LogisticRegression()

# Separando os dados para treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=16)

# fazendo balanceamento dos arquivos e treinando o modelo
kfold = KFold(n_splits=10)
for idx, (idx_treino, idx_validacao) in enumerate(kfold.split(X_train)):

  X_split_treino = X_train.iloc[idx_treino, :]
  y_split_treino = y_train.iloc[idx_treino, :]

  sm = SMOTE(random_state=16)
  X_split_treino, y_split_treino = sm.fit_resample(X_split_treino, y_split_treino)

  lr.fit(X_split_treino, y_split_treino.values.flatten())

# exportando o modelo
joblib.dump(lr, 'classificar_satisfacao.pkl', compress=9)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

['classificar_satisfacao.pkl']